In [1]:
# Load a list of integrals of Laguerre Gaussians
Lag=load('laguerre_integrals')


##########################################################################################################
# -----------------------------------------------------------------
#  lam_verify
# -----------------------------------------------------------------
#
# INPUT:
# - genus: the genus in which we want to prove a bound
# - param: a pair (Z,W), where:
#          * Z is a list of prescribed double roots for f(sqrt(x))
#          * W is a list of prescribed double roots for f^(sqrt(x))
#
# THE PROGRAM:
# - Computes a Fourier pair (f,f^) based on "genus" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of parameters: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "genus" and "param"    
# - Verifies whether the pair (f,f^) satisfies the conditions of Theorem 6.1
# - If so, gives a rigorous estimate for the bound from that Theorem
#
# OUTPUT:
# - If the conditions are satisfied: the computed bound
# - If not, an error message
##########################################################################################################

def lam_verify(genus,param):
    print('\n\n----------------------------------------------------------------')
    print('lam_verify')
    print('genus: ', genus, '\n\n')
    
    (Z,W) = param
    
    # Make sure the entries of Z an W are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]

    
    m = len(Z) #number of forced double roots for f
    n = len(W) #number of forced double zeros for f^

    if n==0:
        return 'W should not be empty'
    
    
    (u,v) = lam_poly(genus,param)
    u_der = u.derivative()
    u_der_der = u_der.derivative()
    v_der = v.derivative()
    v_der_der = v_der.derivative()

    print('Computed the pair of polynomials (u,v) based on the input')
   
    if v(0)<=0:
        return 'f^ is negative at the origin'
            
    if sgn(u.leading_coefficient())==-1:     # Check that u is eventually positive
        return 'f takes negative values near infinity'

    if sgn(v.leading_coefficient())==1:  # Check that v is eventually negative
        return 'f^ takes positive values near infinity'

    R = my_bisect(v,0,W[0])   # Check that the root of v lies before the first prescribed zero
    if R == Infinity:
        return 'The root lies past the first prescribed 0'
            
    val = R + 0.25
        
    if pari.polsturm(u,[0,Infinity])>m:  # Check that f has no other real roots than those that we prescribe
        return 'f has an extra zero'
                
    if pari.polsturm(v,[R,Infinity])>n: # Check that f^ has no other real roots than those that we prescribe
        return 'f^ has an extra zero'

    # Check that u and v vanish at the prescribed roots
    for j in range(m):
        if u(Z[j])!=0 :
            return 'f does not vanish at one of the prescribed zeros'
                        
        if u_der(Z[j])!=0 :
            return 'One of the prescribed zeros of f is not double'

        if u_der_der(Z[j])<=0 :
            return 'The second derivative of f at one of the double roots is non-positive'

    for j in range(n):
        if v(W[j])!=0 :
            return 'f^ does not vanish at one of the prescribed zeros'
                        
        if v_der(W[j])!=0 :
            'One of the prescribed zeros of f^ is not double'        

        if v_der_der(W[j])>=0 :
            return 'The second derivative of f^ at one of the double roots is non-negative'

    print('Verified the conditions on the signs of f and f^')

    
    # Compute an estimate for the integral
    # Below, RBF and CBF denote the RealBallField and ComplexBallField from the Arb package
    
    # h is defined so that f^(x) = h(x^2)
    def h(t):
        return v(t)*e^(-t/2)
    
    # We estimate the integral term in the Selberg trace formula
    truncated_integral = 2*(genus-1)*CBF.integral(lambda x, _: h(x^2)*x*tanh(CBF(pi)*x),0,200)
    
    print('Computed the truncated integral')
    
    # For the remainder of the integral, we compute an indefinite integral
    # We will use that x^(2*n+1) * exp(-x^2 / 2) admits a primitive
    # of the form p_n(x)*exp(-x^2 / 2), where p_n is a polynomial
    
    # First we compute the p_n's:
    max_d = v.degree() # the maximal degree for which we need to know p_n
    p = [-1] # initializing p[0] (which is p_1)
    for n in range(1,max_d):
        p += [-x^(2*n)+2*n*p[n-1]] # adding p_n to the list, the recurrence we use is obtained
                                   # from integration by parts
    
    # Compute the primitive of x*v(x^2)*exp(-x^2/2)
    V = 0
    for n in range(max_d):
        V += v.coefficients()[n]*p[n]
        
    remainder = -2*(genus-1)*V.subs(x==200)*exp(-200^2/2)*tanh(pi*200)
    
    print('Computed the remainder term of the integral')
    
    # Verify that f^(i/2) <= 2*(genus-1)*integral( f^(y)*y*tanh(pi*y) dy, 0, infinity )
    if not RBF(h(-1/4)) <= RBF(truncated_integral) + RBF(remainder):
        return 'The integral inequality is not satisfied'   
    
    print('\n\nBound on lambda_1: ',val)
    
    return val  





##########################################################################################################
# -----------------------------------------------------------------
#  my_bisect
# -----------------------------------------------------------------
# INPUT:
# - f: a function of a single variable
# - a: a lower bound for the root of f
# - b: an upper bound for the root of f
# 
# THE PROGRAM:
# - Computes an approximation for a root of f using the bisection method.
#   The program is set up in such a way that the returned value lies to the right of the root.
#
# OUTPUT:
# The approximation for the root
##########################################################################################################

def my_bisect(f,a,b,tol=1e-13):
    
    if f(a)<0 or f(b)>0:
        return Infinity
    
    while b-a > tol: 
        c = (a+b)/2
        if f(c)>=0:
            a = c
        else:
            b = c
            
    return b

##########################################################################################################
# -----------------------------------------------------------------
#  lam_poly
# -----------------------------------------------------------------
# INPUT:
# - genus: the genus in which we want to prove a bound
# - param: a pair (Z,W), where:
#          * Z is a set of prescribed double roots for f
#          * W is a set of prescribed double roots for f^
# 
# THE PROGRAM:
# - Computes a pair (u,v) of polynomials based on "genus" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of roots: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "genus" and "param"    
#
# OUTPUT:
# (u,v)
##########################################################################################################

def lam_poly(genus,param):

    (Z,W) = param
    
    # Make sure the entries of Z an W are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    
    m = len(Z) #number of forced double roots for f
    n = len(W) #number of forced double zeros for f^
    if n==0:
        print('W should not be empty')
        return
    c = 2+2*(m+n) #number of conditions imposed

    # Compile a list of Laguerre polynomials and their derivatices
    P.<x> = PolynomialRing(QQ) 
    L = [ P(gen_laguerre(n,-1/2,x)) for n in range(c)]
    L_der = [ L[n].derivative() for n in range(c)]
    
    # Load the integrals of the Laguerre Gaussians that we computed beforehand.
    # The correctness of these integrals does not need to be verifiable, they are only used to set linear
    # condition on v and don't influence whether or not u(x^2)*exp(-x^2/2) and v(y^2)*exp(-y^2)/2
    # are each other's Fourier transforms.
    # The fact that the pair (u,v) satisfies all the necessary conditions is verified a posteriori in
    # lam_verify.
    I = Lag[0:c]
    
    A = [[L[j](0) for j in range(c)]]
    
    B = [[(-1)^j*L[j](W[k]) for j in range(c)] for k in range(n)] #for f^ to vanish at the w_k
    C = [[(-1)^j*L_der[j](W[k]) for j in range(c)] for k in range(n)] #for f^ to vanish at the w_k
    
    D = [[L[j](Z[k]) for j in range(c)] for k in range(m)] #for f to vanigh at the z_k
    E = [[L_der[j](Z[k]) for j in range(c)] for k in range(m)] #for f' to vanish at the z_k
    F = [[(-1)^j*(L[j](-1/4)*RDF(e^(1/8))-0.9999*2*(genus-1)*I[j]) for j in range(c)]] #for f^(i/2) to equal the integral

    M = pari(matrix(QQ,A+B+C+D+E+F)) #matrix of values of the Laguerre polynomials and their derivative or combinations thereof
    Y = pari(matrix(QQ,[1]+[0]*(c-1)).transpose()) #this is the vector of target values
    X = vector(pari.matinverseimage(M,Y).sage())

    u = sum([X[j]*L[j] for j in range(c)]) #the polynomial for g 
    v = sum([(-1)^j*X[j]*L[j] for j in range(c)]) #the polynomial for h
    
    return (u,v)

In [2]:
# Verify Table 3

# Load the lambda_1 parameters
lam_parameters = load('parameters_lambda')

# Compile a list of systole bounds in genus 2 - 20
lam_list = []
for genus in range(2,21):
    param = lam_parameters[genus-2]
    lam_list += [lam_verify(genus,param)]

lam_list



----------------------------------------------------------------
lam_verify
genus:  2 


Computed the pair of polynomials (u,v) based on the input
Verified the conditions on the signs of f and f^
Computed the truncated integral
Computed the remainder term of the integral


Bound on lambda_1:  4.62530602471902


----------------------------------------------------------------
lam_verify
genus:  3 


Computed the pair of polynomials (u,v) based on the input
Verified the conditions on the signs of f and f^
Computed the truncated integral
Computed the remainder term of the integral


Bound on lambda_1:  2.81642644989433


----------------------------------------------------------------
lam_verify
genus:  4 


Computed the pair of polynomials (u,v) based on the input
Verified the conditions on the signs of f and f^
Computed the truncated integral
Computed the remainder term of the integral


Bound on lambda_1:  2.17380546349112


-----------------------------------------------------------

[4.62530602471902,
 2.81642644989433,
 2.17380546349112,
 1.83676574559572,
 1.62559553854708,
 1.48000724682768,
 1.37280388969602,
 1.28902305944237,
 1.22218850638080,
 1.16816876279107,
 1.12232610976439,
 1.08325956790375,
 1.04921652195410,
 1.01800468053847,
 0.991734205112135,
 0.968259277781836,
 0.947179324929155,
 0.928090590802994,
 0.911389160852995]